In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import pickle
from glob import glob

files = sorted(glob('/home/husein/processed-youtube-asr-whisper-large-v2/*.pkl'))
len(files)

790

In [3]:
from datasets import Audio
from tqdm import tqdm
import numpy as np
import IPython.display as ipd

In [4]:
sr = 16000
reader = Audio(sampling_rate = sr)

In [5]:
with open(files[0], 'rb') as fopen:
    data = pickle.load(fopen)

In [6]:
max(data[0]['asr_model'][0], key=data[0]['asr_model'][0].get)

'ms'

In [7]:
data[0]

{'wav_data': "/home/ubuntu/processed-youtube-asr-whisper-large-v2/'avin_a_bit_of_a_Frank_&_Pat_with_London_Boy_Mark_O'dea_(@markodea8)_[pJnuqq7oAeY]/0.mp3",
 'start': 0.0,
 'asr_model': ({'br': 4.848451499128714e-05,
   'ka': 8.601645276939962e-06,
   'sn': 0.00014158668636810035,
   'ms': 0.945112943649292,
   'sa': 6.63459868519567e-05,
   'yi': 5.955907454335829e-06,
   'so': 2.8467734409787226e-06,
   'pa': 7.193873170763254e-05,
   'bg': 5.460308966576122e-05,
   'si': 0.00020463635155465454,
   'hi': 0.0002003276313189417,
   'mi': 0.00033299013739451766,
   'jw': 0.006487382110208273,
   'bo': 0.00010008537356043234,
   'sr': 4.6827876758470666e-06,
   'haw': 0.0004717360425274819,
   'tr': 0.0009716611821204424,
   'sq': 1.6484309526276775e-05,
   'kk': 1.390101169818081e-05,
   'eu': 1.4503937563858926e-05,
   'tg': 9.537285222904757e-06,
   'da': 0.00022655694920103997,
   'pl': 3.691731035360135e-05,
   'id': 0.0038747312501072884,
   'ru': 0.0002895525249186903,
   'ps': 8.

In [8]:
ms_score = [s['avg_logprob'] for s in data[0]['asr_model'][1]['segments']]
en_score = [s['avg_logprob'] for s in data[0]['asr_model'][2]['segments']]

if np.mean(ms_score) > np.mean(en_score):
    selected = data[0]['asr_model'][1]['segments']
else:
    selected = data[0]['asr_model'][2]['segments']

In [9]:
i = 0

s = selected[i]['start']
e = selected[i]['end']
t = selected[i]['text']
s, e, t

(0.0, 9.0, ' The Fashion Week.')

In [10]:
f = data[0]['wav_data'].replace('/home/ubuntu/', '/home/husein/')
audio = reader.decode_example(reader.encode_example(f))['array']
y = audio[int(s * sr): int(e * sr)]

In [11]:
ipd.Audio(y, rate = sr)

In [12]:
import malaya_speech

`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.


In [14]:
import malaya
from malaya.text.normalization import cardinal

tokenizer = malaya.tokenizer.Tokenizer(hypen = False, parliament = False, time = False, time_pukul = False,
                                      temperature = False, distance = False, volume = False, duration = False,
                                      weight = False, date = False, money = False)

In [15]:
import unicodedata
import re
import itertools

vocabs = [" ", "a", "e", "n", "i", "t", "o", "u", "s", "k", "r", "l", "h", "d", "m", "g", "y", "b", "p", "w", "c", "f", "j", "v", "z", "0", "1", "x", "2", "q", "5", "3", "4", "6", "9", "8", "7"]

def preprocessing_text(string):
    
    tokenized = tokenizer.tokenize(string)
    string = ' '.join(tokenized)
    string = unicodedata.normalize('NFC', string.lower())
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [20]:
directory = '/home/husein/ssd3/stt/whisper-v2'

In [19]:
directory = '/home/husein/ssd3/stt-v2/whisper'

In [20]:
!mkdir {directory}

In [21]:
import torchaudio
import torch
import os

In [27]:
dataset = []
index = 0
for no, f in enumerate(files):
    
    with open(f, 'rb') as fopen:
        data = pickle.load(fopen)
    
    for i in tqdm(range(len(data))):
        lang = max(data[i]['asr_model'][0], key=data[i]['asr_model'][0].get)
        if lang not in {'ms', 'en'}:
            continue
        
        ms_score = [s['avg_logprob'] for s in data[i]['asr_model'][1]['segments']]
        en_score = [s['avg_logprob'] for s in data[i]['asr_model'][2]['segments']]

        if np.mean(ms_score) > np.mean(en_score):
            selected = data[i]['asr_model'][1]['segments']
        else:
            selected = data[i]['asr_model'][2]['segments']
            
        audio = reader.decode_example(reader.encode_example(data[i]['wav_data'].replace('/home/ubuntu/', '/home/husein/')))['array']
        for k in range(len(selected)):
            s = selected[k]['start']
            e = selected[k]['end'] + 0.1
            t = selected[k]['text']
            cleaned_t = preprocessing_text(t)
            if not len(cleaned_t):
                continue
                
            audio_path = os.path.join(directory, f'{index}.mp3')
            a = audio[int(s * sr): int(e * sr)]
            torchaudio.save(audio_path, 
                            torch.tensor(a.astype('float32')).unsqueeze(0), 
                            16000, format='mp3')

            dataset.append(
                {
                    'file': audio_path,
                    'text': t,
                    'cleaned': cleaned_t,
                    'no': no,
                    'i': i,
                    'k': k,
                    'avg_logprob': selected[k]['avg_logprob'],
                    'no_speech_prob': selected[k]['no_speech_prob'],
                }
            )
            index += 1

 20%|██████████████████                                                                          | 42/214 [00:08<00:33,  5.09it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 163/163 [00:31<00:00,  5.21it/s]


 69%|██████████████████████████████████████████████████████████████▊                            | 160/232 [00:29<00:12,  5.59it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 249/249 [00:46<00:00,  5.37it/s]


 50%|█████████████████████████████████████████████▋                                             | 167/333 [00:31<00:33,  4.93it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 59%|██████████████████████████████████████████████████████▏                                    | 229/385 [00:43<00:28,  5.40it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|█████████████████████████████████████████████████████████████████████████████████████

  2%|██                                                                                           | 7/318 [00:01<01:00,  5.11it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 78%|██████████████████████████████████████████████████████████████████████▋                    | 241/310 [00:46<00:12,  5.36it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [24]:
len(dataset)

209